From https://docs.dgl.ai/en/0.6.x/tutorials/blitz/1_introduction.html

In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from dgl.data import DGLDataset

Using backend: pytorch


In [2]:
edges = pd.read_csv('../data/twitch_gamers/large_twitch_edges.csv', header=0)
edges.columns = ['source', 'target']
features = pd.read_csv('../data/twitch_gamers/large_twitch_features.csv', header=0)

In [3]:
features

,views,mature,life_time,created_at,updated_at,numeric_id,dead_account,language,affiliate
0,7879,1,969,2016-02-16,2018-10-12,0,0,EN,1
1,500,0,2699,2011-05-19,2018-10-08,1,0,EN,0
2,382502,1,3149,2010-02-27,2018-10-12,2,0,EN,1
3,386,0,1344,2015-01-26,2018-10-01,3,0,EN,0
4,2486,0,1784,2013-11-22,2018-10-11,4,0,EN,0
...,...,...,...,...,...,...,...,...,...
168109,4965,0,810,2016-07-20,2018-10-08,168109,0,EN,0
168110,4128,1,2080,2013-01-31,2018-10-12,168110,0,EN,0
168111,3545,0,1797,2013-11-08,2018-10-10,168111,0,EN,1
168112,892736,1,2135,2012-12-07,2018-10-12,168112,0,EN,0


In [4]:
features['created_at'] = pd.to_datetime(features['created_at'])
features['updated_at'] = pd.to_datetime(features['updated_at'])

features['delta_days'] = (features['updated_at'] - features['created_at']).dt.total_seconds()/(60*60*24)

features['language'] = pd.factorize(features['language'])[0]

In [5]:
class TwitchGamersDataset(DGLDataset):
    def __init__(self):
        super().__init__(name='twitch_gamers')

    def process(self):
        nodes_data = pd.read_csv('../data/twitch_gamers/large_twitch_features.csv', header=0)
        edges_data = pd.read_csv('../data/twitch_gamers/large_twitch_edges.csv', header=0)

        nodes_data['created_at'] = pd.to_datetime(nodes_data['created_at'])
        nodes_data['updated_at'] = pd.to_datetime(nodes_data['updated_at'])
        nodes_data['delta_days'] = (nodes_data['updated_at'] - nodes_data['created_at']).dt.total_seconds()/(60*60*24)
        nodes_data['language'] = pd.factorize(nodes_data['language'])[0]

        node_features = torch.from_numpy(nodes_data[['views', 'mature', 'life_time', 'delta_days', 'dead_account', 'language']].to_numpy()).float()
        node_labels = torch.from_numpy(nodes_data['affiliate'].astype('category').cat.codes.copy().to_numpy())
        edges_src = torch.from_numpy(edges_data['numeric_id_1'].to_numpy())
        edges_dst = torch.from_numpy(edges_data['numeric_id_2'].to_numpy())

        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=nodes_data.shape[0])
        self.graph.ndata['feat'] = node_features
        self.graph.ndata['label'] = node_labels

        # If your dataset is a node classification dataset, you will need to assign
        # masks indicating whether a node belongs to training, validation, and test set.
        n_nodes = nodes_data.shape[0]
        n_train = int(n_nodes * 0.6)
        n_val = int(n_nodes * 0.2)
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train:n_train + n_val] = True
        test_mask[n_train + n_val:] = True
        self.graph.ndata['train_mask'] = train_mask
        self.graph.ndata['val_mask'] = val_mask
        self.graph.ndata['test_mask'] = test_mask

    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1

dataset = TwitchGamersDataset()
graph = dataset[0]

print(graph)

Graph(num_nodes=168114, num_edges=6797557,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int8), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})


C:\Users\Peter\AppData\Local\Temp/ipykernel_12848/506360704.py:15: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:178.)
  node_labels = torch.from_numpy(nodes_data['affiliate'].astype('category').cat.codes.to_numpy())


In [6]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(graph.ndata['feat'].shape[1], 16, 2)

In [7]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask].long())

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 10 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))


graph = dgl.add_self_loop(graph)
train(graph, model)

C:\Users\Peter\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch 0, loss: 88574.445, val acc: 0.514 (best 0.514), test acc: 0.513 (best 0.513)
In epoch 10, loss: 19614.129, val acc: 0.486 (best 0.514), test acc: 0.487 (best 0.513)
In epoch 20, loss: 4804.613, val acc: 0.487 (best 0.514), test acc: 0.488 (best 0.513)
In epoch 30, loss: 3541.513, val acc: 0.514 (best 0.514), test acc: 0.513 (best 0.513)
In epoch 40, loss: 791.626, val acc: 0.496 (best 0.514), test acc: 0.498 (best 0.513)
In epoch 50, loss: 760.701, val acc: 0.510 (best 0.514), test acc: 0.509 (best 0.513)
In epoch 60, loss: 115.730, val acc: 0.474 (best 0.514), test acc: 0.472 (best 0.513)
In epoch 70, loss: 4135.557, val acc: 0.514 (best 0.514), test acc: 0.513 (best 0.513)
In epoch 80, loss: 2678.466, val acc: 0.514 (best 0.514), test acc: 0.513 (best 0.513)
In epoch 90, loss: 3985.451, val acc: 0.514 (best 0.514), test acc: 0.513 (best 0.513)
